In [3]:
P = Primes()
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
C.<x> = PolynomialRing(CC)
S = PowerSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.
def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])
def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer
def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))
def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))
def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)
def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))
def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)
def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)
def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.
def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)
def Jcoefficient(n,m,x):
    f = J(n+2,m,x)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]
def xjNewStep(n,m,x): 
    f=J(n,m,x)
    fl=f.list()
    return (sum(c*x**i*(m**3*2**6)**i for i,c in enumerate(fl)))
def xjNew(n,m,x):
    return xjNewStep(n-1,m,x)
def xjNewCoefficient(n,m,x):
    if n>0:return (xjNew(n,m,x)-xjNew(n-1,m,x))/x^n
    if n==0:return 1
def xjNewTest(n,m,x):
    return D(xjNewStep(n-1,m,x))
def polynomialCoefficient(n,poly):
    return poly.list()[n]
def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent
def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)
def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())
def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

def prime_n(a):return P.unrank(a-1)

def primeFactors(n):
    answer=[]
    lf=list(n.factor())
    length=len(lf)
    for k in [0..length-1]:
        pair=lf[k]
        (base,exponent)=pair
        answer=answer+[[base]]
    return(flatten(answer))

def primeFactorsInList(lst):
    ld=len(lst)
    primelist=[]
    for k in [0..ld-1]:
        a=ZZ(lst[k])
        primelist=primelist+[primeFactors(a)]
    primelist=flatten(primelist)
    primelist=list(set(primelist))
    return primelist

def den(a):
    if a in QQ:
        if a in ZZ:return 1
        return a.denominator()
def listdens(lst):
    return list(den(a) for a in lst)
def polynomialdenominator(poly):return lcm(listdens(poly))
def polynomialnumerator(poly):
    return expand(poly*polynomialdenominator(poly))
def num(a):return a.numerator()
def prime(n):
    P=Primes()
    return P.unrank(n-1)
def inclusiveListPrimesTo(n):
    if is_prime(n):
        return list(primes(n+1))
    else:
        return list(primes(n))
def chunk(k,list):
    chnk=[]
    for n in [k..len(list)-1]:
        chnk=chnk+[list[n]]
    return chnk
def drop(m,lst): return list(Set(lst).difference({m}))
def sort(list):
    answr=[]
    chnk=list
    for n in [0..len(list)-1]:
        m = min(chnk)
        answr=answr+[m]
        chnk=drop(m,chnk)
    return answr
def ord(p,n):return QQ.valuation(p)(n)
def base_conversion(number, base_from, base_to):  # by Vishnu Namboothiri
# https://ask.sagemath.org/question/9657/how-to-output-decimal-numbers-in-different-radix/
    if(base_from==10):
        return stripQuotationMarks((number.str(base=base_to)))
    else:
        decimal = Integer(str(number), base=base_from)
        return (stripQuotationMarks(decimal.str(base=base_to)))
def lastFactorWithIntegralFactors(polynomial):
    return polynomialnumerator(lastfactor(polynomial))
def numericalFactorWithIntegralFactors(polynomial):
    nf=numericalfactor(polynomial);
    pd=polynomialdenominator(lastfactor(polynomial))
    return nf/pd


def primeDivisors(n):
    pd=[]
    n=n
    fn=factor(n)
    ln=len(fn)
    for k in [0..ln-1]:
        part = fn[k]
        (prime,exponent) = part
        pd = pd + [prime]
    return pd 
def fOverField(f,prime,power):
    answer=[]
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    ppoly = U(f)
    expr=str(ppoly)
    var('x')
    fp=ppoly.factor()
    return ppoly

###################### following two routines disagree ####################
def factorOverField(f,prime,power):
    answer=[]
    T = GF(prime^power,'t')
    G.<t>= GF(prime^power)
    U.<x> = T[]
    ppoly = U(f)
    expr=str(ppoly)
    var('x')
    fp=ppoly.factor()
    return fp

def factorOverField2(poly,prime,power,j):
    sd = leastSplittingDegree(poly,prime,bound)
    T=GF(prime^j)
    G.<t>= GF(prime^sd)
    U.<x> = T[]
    ppoly=U(poly)
    fpp=ppoly.factor()
    return fpp

############################

#######################################################
# splitting tests below fail at  e.g. f = x + 1 
############################################################

def splitsTF(f,prime,power):
    T = GF(prime^power)
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx > 1:return(False)
    return(True)

def leastSplittingDegree(f,prime,bound):
    for power in [1..bound]:
        tf=splitsTF(f,prime,power)
        if tf == True:return power
    return -1

#########################################################

def firstNprimes(n):
    P=Primes()
    first=[]
    for k in [0..n-1]:
        first=first+[P.unrank(k)]
    return first

def slopeExistsTF(pairlist):
    lenth=len(pairlist)
    if lenth<3:return False
    slopes=[]
    for k in [1..lenth-1]:
        pair1=pairlist[k-1]
        abscissa1=pair1[0]
        ordinate1=pair1[1]
        pair2=pairlist[k]
        abscissa2=pair2[0]
        ordinate2=pair2[1]
        dx=abscissa2-abscissa1
        if dx == 0:return False
        dy=ordinate2-ordinate1
        slope=dy/dx
        slopes=slopes+[slope]
    slopes=Set(slopes)
    if len(slopes) == 1:
        return [True,slopes[0]]
    if len(slopes) > 1:
        return False 
    
def zeroOfLinearMonomial(monomial,x):return x-monomial

def totalDegreeOverField(f,prime,power):
    T = GF(prime^power,'t')
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    td=0
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        td=td+exponent
    return td

def dropzeros(list):
    data=[]
    el=len(list)
    for j  in [0..el-1]:
        lj=list[j]
        if lj != 0:data=data+[lj]
    return data

def numericalTermOverFieldPrint(f,prime,power):
    T = GF(prime^power,'t')
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    print gf
    lnf = len(gf)-1
    xproduct=1
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        xproduct=xproduct*monomial^exponent
    return g/xproduct


def numericalTermOverFieldNoPrint(f,prime,power):
    T = GF(prime^power,'t')
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    xproduct=1
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        xproduct=xproduct*monomial^exponent
    return g/xproduct

def factoredCoercionFOverField(f,prime,power):
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    g = U(f)
    gf = g.factor()
    return gf

def unfactoredCoercionFOverField(f,prime,power):
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    g = U(f)
    return g

In [ ]:
## printed output has been snipped

import pickle
wfile=open('/Users/barrybrent/run1jun21no1.txt','a')
data=[]
for m in [3..500]:
    print m
    data=data+[[m,normalJ(200,m)]]
s=pickle.dumps(str(data))
wfile.write(s)
wfile.close()
#########################
import pickle
rfile = open('/Users/barrybrent/run1jun21no1.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)

polydata=[]

wfile = open('/Users/barrybrent/run14jun21no14.txt','a') # xJ polynomials
for n in [0..200]:
    data=[]
    for k in [0..2*n+1]:
        print (n,k)
        m = s[k][0]
        poly=x*s[k][1] 
        cf=polynomialCoefficient(n,poly)
        data=data+[[m,cf*m^(2*n)]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[n-1,rl]] 

    
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close() # print-output snipped

In [3]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
print len(s)
print s[1]

201
[0, 3/8*x^2 + 1/2]


In [4]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=12
for k in [1..6]:
    index=s[k][0]
    poly = s[k][1]
    print poly

3/8*x^2 + 1/2
69/1024*x^4 - 1/128*x^2 - 3/64
1/128*x^6 - 29/864*x^4 + 1/216*x^2 + 1/54
5601/8388608*x^8 - 3821/524288*x^6 + 5195/262144*x^4 - 101/32768*x^2 - 303/32768
3/65536*x^10 - 16367/18432000*x^8 + 7961/1382400*x^6 - 21809/1728000*x^4 + 373/172800*x^2 + 373/72000
23003/8589934592*x^12 - 2254159/28991029248*x^10 + 338577733/391378894848*x^8 - 106154827/24461180928*x^6 + 68420351/8153726976*x^4 - 4754693/3057647616*x^2 - 4754693/1528823808


In [5]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=12
for k in [1..6]:
    index=s[k][0]
    poly = s[k][1]
    print polynomialnumerator(D(poly))

3*x^2 + 4
69*x^4 - 8*x^2 - 48
27*x^6 - 116*x^4 + 16*x^2 + 64
5601*x^8 - 61136*x^6 + 166240*x^4 - 25856*x^2 - 77568
10125*x^10 - 196404*x^8 + 1273760*x^6 - 2791552*x^4 + 477440*x^2 + 1145856
16769187*x^12 - 486898344*x^10 + 5417243728*x^8 - 27175635712*x^6 + 52546829568*x^4 - 9737611264*x^2 - 19475222528


In [5]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
prime=3
count=0
data=[]
for k in [1..12]:
    n=s[k][0]
    poly = s[k][1]
    poly=polynomialnumerator(D(poly))
    sd=leastSplittingDegree(poly,prime,bound)
    coercedpoly=unfactoredCoercionFOverField(poly,prime,sd)
    print "------------------------------------------------------------------------------"
    print "n: ",n," prime: ",prime
    print "numerator:";print poly;print;print "coerced numerator:";print;print coercedpoly

------------------------------------------------------------------------------
n:  0  prime:  3
numerator:
3*x^2 + 4

coerced numerator:

1
------------------------------------------------------------------------------
n:  1  prime:  3
numerator:
69*x^4 - 8*x^2 - 48

coerced numerator:

x^2
------------------------------------------------------------------------------
n:  2  prime:  3
numerator:
27*x^6 - 116*x^4 + 16*x^2 + 64

coerced numerator:

x^4 + x^2 + 1
------------------------------------------------------------------------------
n:  3  prime:  3
numerator:
5601*x^8 - 61136*x^6 + 166240*x^4 - 25856*x^2 - 77568

coerced numerator:

x^6 + x^4 + x^2
------------------------------------------------------------------------------
n:  4  prime:  3
numerator:
10125*x^10 - 196404*x^8 + 1273760*x^6 - 2791552*x^4 + 477440*x^2 + 1145856

coerced numerator:

2*x^6 + 2*x^4 + 2*x^2
------------------------------------------------------------------------------
n:  5  prime:  3
numerator:
16769

In [2]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
prime=3
count=0
data=[]
for k in [1..12]:
    n=s[k][0]
    poly = s[k][1]
    poly=polynomialnumerator(D(poly))
    sd=leastSplittingDegree(poly,prime,bound)
    nt=numericalTermOverFieldNoPrint(poly,prime,sd)
    print "--------------------------------------------------------------------------------"
    print "n: ",n," numerical term: ",nt
    print 
    

([1, 1, 1, 1, 2, 1, 2, 1, 2, 2], 10)
